# Исследование надежности заемщиков

Входные данные от банка — статистика о платёжеспособности клиентов.

Задача определить влияет ли семейное положение, количество детей, уровень дохода и цель кредита на факт погашения кредита в срок.

## Обзор данных

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов. Названия колонок без нарушения стиля.
Имеем описание данных:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита
   
   Что не так:
1. Количество значений в 'days_employed' (19351) и 'total_income' (19351) отличается от остальных (21525):
10% - пропущенные значения (разница между 21525 и 19351, в %). Количество одинаковое (по 19351). Гипотеза: возможно пропуски в одних и тех же строках. 
2. Значения в колонке общий трудовой стаж в днях (days_employed) присутствуют отрицательные. Это артефакты, человеческий фактор. Гипотеза: это могло быть желание поставить тире при внесении данных.
3. Типы данных различны в столбцах: float64(2), int64(5), object(5). Имеет смысл перевести в колонках days_employed и total_income тип данных из float в int.
4. В колонке education присутствуют значения, записанные с использованием заглавных и строчных букв.
5. Проверить остальные столбцы на дубликаты и артефакты. 

## Заполнение пропусков

In [4]:
df[df['days_employed'].isna()].head(10) # проверка на пропуски

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


    Гипотеза подтверждается - пропуски в колонках days_employed и total_income на одних и тех же строчках. 
    Значит ошибка техническая, ошибка записи данных или ошибка выгрузки данных. 
    Можно сказать разработчикам, т.к. 10 % пропусков это много.
    Попробуем своими силами заполнить пропуски. Но сначала избавимся от отрицательных значений в days_employed.

In [5]:
df['days_employed'] = df['days_employed'].abs()  # применим функцию abs(), возвращает абсолютную величину. 

In [6]:
df.head() # отрицательные значения стали положительными

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [7]:
print('Медиана "days_employed":', df['days_employed'].median(), ' В годах:', df['days_employed'].median() / 365)
print('Среднее значение "days_employed":', df['days_employed'].mean(), ' В годах:', df['days_employed'].mean() / 365)
print('Минимум "days_employed":', df['days_employed'].min(), ' В годах:', df['days_employed'].min() / 365)
print('Максимум "days_employed":', df['days_employed'].max(), ' В годах:', df['days_employed'].max() / 365)
# проверим значения колонки "трудовой стаж в днях", определим описательные метрики

Медиана "days_employed": 2194.220566878695  В годах: 6.0115631969279315
Среднее значение "days_employed": 66914.72890682236  В годах: 183.32802440225305
Минимум "days_employed": 24.14163324048118  В годах: 0.06614146093282515
Максимум "days_employed": 401755.40047533  В годах: 1100.6997273296713


    1. В колонке "days_employed" - ошибка, не ясна единица измерения.
    2. Аномально высокие значения в столбце days_employed можно разделить на 24 (есть гипотеза, что эти значения получились из-за того, что стаж указан в часах) и тогда получится вполне себе адекватный стаж (но имеет смысл уточнить могла ли быть такая ошибка при выгрузке/заполнении данных)
    3. Гораздо важнее сообщить об ошибке разработчикам, для выяснения на каком этапе происходит сбой и устранения ошибки выдачи некорректной информации.

Заполним пропуски NAN в столбце 'total_income' медианным значением, в зависимости от типа занятости.

    Медиана является более объективной информацией при определении дохода, чем среднее значение.
    Ровно половина выборки имеет доход меньше медианного значение и ровно половина больше.

In [8]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))
   # заменим пропуски медианой, в зависимости от типа занятости (этот параметр сильнее всего влияет на уровень зарплаты)
   # с помощью функции transform

In [9]:
df[df['total_income'].isna()].head(10)  # проверим еще раз на пропуски, результат - пропусков нет

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


## Проверка данных на аномалии и исправления

Проверим на аномалии колонку Количество детей

In [10]:
df['children'].value_counts() # определим количество уникальных значений колонки 'children' 

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

    Есть аномалии.
    Значение -1, опечатка, заменим на 1.
    Значение 20 детей, это маловероятно, 0 приписали ошибочно, заменим на 2.
    Количество опечаток не велико, значит ошибка ввода данных (человеческий фактор).

In [11]:
df['children'] = df['children'].abs() # функция abs вернет значения по модулю 

In [12]:
df['children'] = df['children'].replace(20, 2) # и метод replace для замены 20 на 2

In [13]:
df['children'].value_counts() # проверим еще раз уникальные значение, все ок

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [14]:
df['dob_years'].sort_values().value_counts() # проверим колонку возраст заемщика 

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

    Все значения возраста в рамках корректности, кроме нулевых значений. Заменять средним не будем, нулевых значений меньше 1 %, это не критично.
    Просто посмотрим на среднее значение возраста.

In [15]:
dob_years_mean = df['dob_years'].mean()
print('Медианное значение возраста:', df['dob_years'].median())
print('Средний возраст заемщика:', df['dob_years'].mean())

Медианное значение возраста: 42.0
Средний возраст заемщика: 43.29337979094077


## Изменение типов данных

После заполнения пропусков на медианное значение в колонке 'total_income' изменим тип данных

In [16]:
df['total_income'] = df['total_income'].astype('int')
# применим метод astype, для перевода значений из вещественного в целочисленный тип данных

In [17]:
df.info() # проверим результат изменений, total_income стал int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


## Удаление дубликатов

In [18]:
df['education'].sort_values().unique() # посмотрим уникальные значения колонки образование

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

In [19]:
df['education'] = df['education'].str.lower() # приведем значения к одному регистру и перезапишем

In [20]:
df['education'].sort_values().unique() # проверим еще раз на уникальные значения

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

    Также проверим остальные столбцы.

In [21]:
df['family_status'].sort_values().unique()

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

In [22]:
df['family_status'] = df['family_status'].str.lower()

In [23]:
df['gender'].sort_values().unique()

array(['F', 'M', 'XNA'], dtype=object)

In [24]:
df[df['gender'] ==  'XNA']  # проверим что это XNA

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


    Одна строчка со значением XNA. Это не критично, просто заменим ее например, на M - мужской пол.

In [25]:
df['gender'] = df['gender'].replace('XNA','M') # применим метод replace

In [26]:
df['gender'].sort_values().unique()  # пол XNA больше нет в данных 

array(['F', 'M'], dtype=object)

In [27]:
df['income_type'].sort_values().unique()  # проверим тип занятости

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

В колонке Тип занятости все в порядке со значениями.

In [28]:
df.duplicated().sum() # теперь проверим датафрейм на явные дубликаты 
df[df.duplicated()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
3290,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
4182,1,NaN,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,NaN,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
6312,0,NaN,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594,строительство жилой недвижимости
7808,0,NaN,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
7921,0,NaN,64,высшее,0,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
7938,0,NaN,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8583,0,NaN,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование


В датафрейме есть строки-дубликаты (количество 71), у которых полностью совпадают значения в ячейках.

Применим метод drop_duplicates() и метод reset_index(), которые удалят явные дубликаты и проставят новую индексацию. 

In [29]:
df = df.drop_duplicates().reset_index(drop=True)

In [30]:
df.info()  # проверим как изменился датафрейм 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  int64  
 11  purpose           21454 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


    Провела проверку и очистку датафрейма на явные и не явные дубликаты, артефакты и аномалии.
    Количество строк уменьшилось, 21454.
    

## Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, что облегчит работу с таблицей:

    - для уровней образование и семейных статусов

In [31]:
education_dict = df[['education','education_id']] # создадим новый датафрейм для уровней образования

In [32]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True) # удалим дубликаты и новый "словарь" готов
education_dict.head()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [33]:
family_status_dict = df[['family_status','family_status_id']] # сделаем тоже самое для семейного статуса

In [34]:
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True) # удалим дубликаты
family_status_dict.head()  # и проверим результат

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


    Теперь можно удалить столбцы 'education' и 'family_status' из основного датафрейма

In [35]:
df = df.drop(['education', 'family_status'], axis=1)


## Категоризация дохода

Классифицируем заемщиков по доходу, созданим колонку total_income_category, где присвоим буквенную категорию.

    Категории такие:
    0–30000 — 'E';
    30001–50000 — 'D';
    50001–200000 — 'C';
    200001–1000000 — 'B';
    1000001 и выше — 'A'.

In [36]:
def total_income_group(income): # применим функцию для создания категорий
    if income <= 30000:
        return 'E'
    elif 30001<=income<=50000:
        return 'D'
    elif 50001<=income<= 200000:
        return 'C'
    elif 200001<=income<= 1000000:
        return 'B'
    return 'A'

df['total_income_category'] = df['total_income'].apply(total_income_group)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


## Категоризация целей кредита

Создадим следующие категории по целям кредита:

    - операции с автомобилем
    - операции с недвижимостью
    - проведение свадьбы
    - получение образования

In [37]:
df['purpose'].sort_values().value_counts() # сначала определим весь набор уникальных целей

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [38]:
def purpose_group(target):   # применим функцию для укрупнения целей кредита 
    if 'образован' in target:   # переберем все встречающиеcя слова и запишем их в условие без падежа и окончания
        return 'получение образования'
    elif 'автомобил' in target:
        return 'операции с автомобилем'
    elif 'свадьб' in target:
        return 'проведение свадьбы'  
    return 'операции с недвижимостью'

df['purpose_category'] = df['purpose'].apply(purpose_group)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [39]:
df.info() # контрольная проверка перед выводами

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21454 non-null  int64  
 1   days_employed          19351 non-null  float64
 2   dob_years              21454 non-null  int64  
 3   education_id           21454 non-null  int64  
 4   family_status_id       21454 non-null  int64  
 5   gender                 21454 non-null  object 
 6   income_type            21454 non-null  object 
 7   debt                   21454 non-null  int64  
 8   total_income           21454 non-null  int64  
 9   purpose                21454 non-null  object 
 10  total_income_category  21454 non-null  object 
 11  purpose_category       21454 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


## Ответы на вопросы.

### Есть ли зависимость между количеством детей и возвратом кредита в срок?
    
    Для этого построим сводную таблицу, с группировками по детям и по возвратам кредита. 

In [40]:
df_pivot_kids = df.pivot_table(index=['children'], 
                               columns='debt', values='gender', aggfunc='count', fill_value=0, margins=True)
df_pivot_kids['ratio_%'] = (df_pivot_kids[1] / df_pivot_kids['All'] * 100).round(1) 
# Определили процент не погасивших кредит в срок по каждой группировке, и методом round округлили
df_pivot_kids.head(10)

debt,0,1,All,ratio_%
children,,,,
0,13028,1063,14091,7.5
1,4410,445,4855,9.2
2,1926,202,2128,9.5
3,303,27,330,8.2
4,37,4,41,9.8
5,9,0,9,0.0
All,19713,1741,21454,8.1


#### Вывод 1:
* Средний процент по выборке, не вернувших кредит в срок, cоставил 8.1 %.
* Разброс отклонений от среднего в группировках по количеству детей максимально составляет - 1.7%.
* Семьи с 5 детьми яляются самыми ответственными в погашении кредитов (100% погашение кредита в срок).
    Конечно, в количестве 9 семей с 5 детьми - это не так много, но стоит задуматься об этой гипотезе "ответственности".
* Процент не погашения кредита в срок у заемщиков без детей самый низкий - 7.5 %.
* Сказать, что есть сильная зависимость погашения кредита в срок от наличия детей и увеличения их количества нельзя. 
    Тем не менее, вероятность погашения кредита в срок, заемщиками без детей выше, но далеко не близка к 100%.
       

### Есть ли зависимость между семейным положением и возвратом кредита в срок?
    
    Построим сводную таблицу, с группировками по семейному статусу и по возвратам кредита. 

In [41]:
df_pivot_status = df.pivot_table(index=['family_status_id'], 
                                columns='debt', values='gender', aggfunc='count', margins=True)
df_pivot_status['ratio_%'] = (df_pivot_status[1] / df_pivot_status['All'] * 100).round(1) 
# Определили процент не погасивших кредит в срок по каждой группировке, и методом round округлили %

df_status = family_status_dict.merge(df_pivot_status, on='family_status_id', how='left') 
# соединим таблицы для наглядности семейного статуса
df_status.head(10)

,family_status,family_status_id,0,1,All,ratio_%
0,женат / замужем,0,11408,931,12339,7.5
1,гражданский брак,1,3763,388,4151,9.3
2,вдовец / вдова,2,896,63,959,6.6
3,в разводе,3,1110,85,1195,7.1
4,не женат / не замужем,4,2536,274,2810,9.8


#### Вывод 2:

Здесь можно сделать определенные выводы о зависимости семейного статуса и факта погашения кредита в срок.
- Люди, состоящие в отношениях, в законном браке (7.5%) более ответственны и кредитонадежны, чем люди в отношениях, 
    но находящиеся в гражданском браке (9.3%).
- Люди, не находящиеся в браке, но ранее были в нем, самые надежные заемщики (6.6% и 7.1%).
- Самая высокая доля, не погасивших кредит в срок (9.8%), у людей не обременных семейными обязательствами.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?    

In [42]:
df_pivot_income = df.pivot_table(index=['total_income_category'], 
                                columns='debt', values='gender', aggfunc='count', margins=True)
df_pivot_income['ratio_%'] = (df_pivot_income[1] / df_pivot_income['All'] * 100).round(1) 
# Определили процент не погасивших кредит в срок
df_pivot_income.head(10)

debt,0,1,All,ratio_%
total_income_category,,,,
A,23,2,25,8.0
B,4686,356,5042,7.1
C,14655,1360,16015,8.5
D,329,21,350,6.0
E,20,2,22,9.1
All,19713,1741,21454,8.1


#### Вывод 3:

Напомню категории:
* выше 1 млн — 'A'
* 200001–1000000 — 'B'
* 50001–200000 — 'C'
* 30001–50000 — 'D'
* 0–30000 — 'E'
  
  
- Категории заемщиков D и E по уровню дохода самые низкодоходные (до 50000 в месяц). 
  Доля этих двух групп незначительна и составляет менее 2 %, т.е. выводы сделать сложно, 
  хотя они и показали самый максимальный (9.1%) и самый минимальный процент (6.0%) в группах из всей выборки.
- Категория 'С' самая многочисленная (16013 из 21454), доля имевших задолженности по кредиту составляет 8.5%, и близка к среднему значению (8,1%).
- Категория 'B' самая ответственная из более доходных заемщиков. % не погасивших, ниже среднего.
- Кредитонадежность заемщика от уровня дохода зависит не сильно.
    

### Как разные цели кредита влияют на его возврат в срок?

In [43]:
df_pivot_purpose = df.pivot_table(index=['purpose_category'], 
                                columns='debt', values='gender', aggfunc='count', margins=True)
df_pivot_purpose['ratio_%'] = ((df_pivot_purpose[1] / df_pivot_purpose['All']) * 100).round(1) 
# Определили процент не погасивших кредит в срок, и методом round округлили
df_pivot_purpose.head(10)

debt,0,1,All,ratio_%
purpose_category,,,,
операции с автомобилем,3903,403,4306,9.4
операции с недвижимостью,10029,782,10811,7.2
получение образования,3643,370,4013,9.2
проведение свадьбы,2138,186,2324,8.0
All,19713,1741,21454,8.1


#### Вывод 4:
    Удивительно, но автокредит является чаще задерживаемым кредитом.
    Получение образования на втором месте с конца по возврату кредита в срок.
    А покупка недвижимости делает людей более кредитонадежными и является самой желанной целью получения кредита (50%).
    Цель кредита может заставить людей делать выбор за что платить в срок.   


## Общий вывод:

**Выводы:**

Среднее значение выборки не возвративших кредит в срок - 8,1%.

Самые интересные социальные результаты по надежности заемщиков получены из группировок по семейному положению и по целям кредитования:
  - Семейный статус сильно влияет на кредитонадежность заемщика. "Еще" не обременные семейными обязательствами являются самыми ненадежными заемщиками - 9.8%. А "уже" не обременные самыми надежными - 6.6%. Статус отношений тоже влияет на кредитонадежность (в законном браке люди более "кредитопослушны", чем в гражданском).  
  - Цель кредита может сделать людей более кредитонадежными заемщиками, так покупка недвижимости из всех целей самая популярная и самая погашаемая в срок. Автокредит может сделать того же заемщика менее кредитонадежным.
  - Количество детей не дает выявить закономерностей, но их отсутствие позволяет сказать, что заемщик самый кредитонадежный и никто не будет отвлекать его от выплаты кредита.
  - Размер дохода не является определяющим критерием для определения надежности заемщика, но похоже может быть самым фактороопределяющим при решении о размере кредита. 
  
Итоговый вывод:
- Клиентом с наибольшей вероятностью просрочки будет человек, который не женат/не замужем, имеющий одного  или несколько детей, который хочет взять кредит на покупку автомобиля, с доходом до 50000 рублей в месяц.
- Портрет самого желанного заемщика будет такой - вдовец или разведенный человек без детей, который берет кредит на покупку недвижимости с доходом от 50 000 до 200 000 рублей в месяц.